# Jass Agent with Reinforcement Learning
### Fact sheet by Ruben Nunez & Jordan Suter

This fact sheet is highly inspired by the paper of Jacop Chapman and Mathias Lechner. <<Deep Q-Learning for Atari Breakout>>

We tried to implement the Jass-Agent similarly as described in the paper.


## Imports

In [19]:
import sys

import numpy as np
import pandas as pd
import tensorflow as tf

from jass.game.game_util import *
from jass.game.game_sim import GameSim
from jass.game.game_observation import GameObservation
from jass.game.const import *
from jass.game.rule_schieber import RuleSchieber
from jass.agents.agent import Agent
from jass.agents.agent_random_schieber import AgentRandomSchieber
from jass.arena.arena import Arena

from tensorflow import keras
from tensorflow.keras import layers

sys.path.append("../deep-jass-project")
# noinspection PyUnresolvedReferences
from agent_gen1 import AgentGen1
# noinspection PyUnresolvedReferences
from agent_gen2 import AgentGen2
# noinspection PyUnresolvedReferences
from agent_gen3 import AgentGen3
# noinspection PyUnresolvedReferences
from agent_gen4 import AgentGen4
# noinspection PyUnresolvedReferences
from agent_gen5 import AgentGen5

In [8]:
# Setup parameter
seed = 42
gamma = 0.99  # Discount factor for past rewards
epsilon = 1.0  # Epsilon greedy parameter
epsilon_min = 0.1  # Minimum epsilon greedy parameter
epsilon_max = 1.0  # Maximum epsilon greedy parameter
epsilon_interval = (epsilon_max - epsilon_min)

# Rate at which to reduce chance of random action being taken
batch_size = 32  # Size of batch taken from replay buffer
max_steps_per_episode = 10000

## Implement the Deep Q-Network
This network learns an approximation of the Q-table, which is a mapping between the states and actions that an agent will take.
In every state are 36 actions, that can eventually be taken. The environment provides the state, and the action is chosen by selecting the larger of the 36 Q-values predicted in the output layer.

In [5]:
num_actions = 36

def create_q_model():
    # Network defined by the Deepmind paper
    inputs = layers.Input(shape=(84, 84, 36,))

    # Convolutions on the frames on the screen
    layer1 = layers.Conv2D(32, 8, strides=4, activation="relu")(inputs)
    layer2 = layers.Conv2D(64, 4, strides=2, activation="relu")(layer1)
    layer3 = layers.Conv2D(64, 3, strides=1, activation="relu")(layer2)

    layer4 = layers.Flatten()(layer3)

    layer5 = layers.Dense(512, activation="relu")(layer4)
    action = layers.Dense(num_actions, activation="linear")(layer5)

    return keras.Model(inputs=inputs, outputs=action)

# The first model makes the predictions for Q-values which are used to
# make a action.
model = create_q_model()
# Build a target model for the prediction of future rewards.
# The weights of a target model get updated every 10000 steps thus when the
# loss between the Q-values is calculated the target Q-value is stable.
model_target = create_q_model()

2021-11-28 01:00:13.861446: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Training process

In [39]:
# In the Deepmind paper they use RMSProp however then Adam optimizer
optimizer = keras.optimizers.Adam(learning_rate=0.00025, clipnorm=1.0) # improves training time

# Experience replay buffers
action_history = []
state_history = []
state_next_history = []
rewards_history = []
done_history = []
episode_reward_history = []
running_reward = 0
episode_count = 0
trick_count = 0

epsilon_random_frames = 50000  # Number of frames to take random action and observe output
epsilon_greedy_frames = 1000000.0  # Number of frames for exploration

# Note: The Deepmind paper suggests 1000000 however this causes memory issues
max_memory_length = 100000 # Maximum replay length
update_after_actions = 1  # Train the model after 1 actions
update_target_network = 10000  # How often to update the target network
loss_function = keras.losses.Huber()  # Using huber loss for stability

## Agent Impl

In [40]:
# Reinforcement Agent Implementation
# by Ruben Nunez & Jordan Suter

class AgentReinforcement(Agent):

    @staticmethod
    def calculate_trump_selection_score(cards, trump: int) -> int:
        trump_score = [15, 10, 7, 25, 6, 19, 5, 5, 5]
        no_trump_score = [9, 7, 5, 2, 1, 0, 0, 0, 0]

        score = 0
        for card in cards:
            suit = int(card / 9)
            exact_card = card % 9
            score += trump_score[exact_card] if trump == suit else no_trump_score[exact_card]

        return score

    episode_reward = 0

    def __init__(self):
        super().__init__()
        self._rule = RuleSchieber() # we need a rule object to determine the valid cards

    def action_trump(self, obs: GameObservation) -> int:
        card_list = convert_one_hot_encoded_cards_to_int_encoded_list(obs.hand)
        threshold = 68
        scores = [0, 0, 0, 0]
        for suit in range(0, 4):
            trump_score = self.calculate_trump_selection_score(card_list, suit)
            scores[suit] = trump_score

        best_score = max(scores)
        best_suit = scores.index(best_score)

        if best_score <= threshold and obs.player < 1:
            return PUSH
        else:
            return best_suit

    def action_play_card(self, obs: GameObservation) -> int:

        global epsilon

        global action_history
        global state_history
        global state_next_history
        global rewards_history
        global done_history
        global episode_reward_history
        global running_reward
        global episode_count
        global trick_count

        trick_count += 1

        # Use epsilon-greedy for exploration
        if trick_count < epsilon_random_frames or epsilon > np.random.rand(1)[0]:
            # Take random action
            action = np.random.choice(num_actions)
        else:
            # Predict action Q-values
            # From environment state
            state_tensor = tf.convert_to_tensor(state)
            state_tensor = tf.expand_dims(state_tensor, 0)
            action_probs = model(state_tensor, training=False)
            # Take best action
            action = tf.argmax(action_probs[0]).numpy()
        # Decay probability of taking random action
        epsilon -= epsilon_interval / epsilon_greedy_frames
        epsilon = max(epsilon, epsilon_min)

        # Apply the sampled action in our environment
        state_next, reward, done, _ = None # env.step(action)
        state_next = np.array(state_next)

        episode_reward += reward

        # Save actions and states in replay buffer
        action_history.append(action)
        state_history.append(state)
        state_next_history.append(state_next)
        done_history.append(done)
        rewards_history.append(reward)
        state = state_next

        # Get indices of samples for replay buffers
        indices = np.random.choice(range(len(done_history)), size=batch_size)

        # Using list comprehension to sample from replay buffer
        state_sample = np.array([state_history[i] for i in indices])
        state_next_sample = np.array([state_next_history[i] for i in indices])
        rewards_sample = [rewards_history[i] for i in indices]
        action_sample = [action_history[i] for i in indices]
        done_sample = tf.convert_to_tensor(
            [float(done_history[i]) for i in indices]
        )

        # Build the updated Q-values for the sampled future states
        # Use the target model for stability
        future_rewards = model_target.predict(state_next_sample)
        # Q value = reward + discount factor * expected future reward
        updated_q_values = rewards_sample + gamma * tf.reduce_max(
            future_rewards, axis=1
        )

        # If final frame set the last value to -1
        updated_q_values = updated_q_values * (1 - done_sample) - done_sample

        # Create a mask so we only calculate loss on the updated Q-values
        masks = tf.one_hot(action_sample, num_actions)

        with tf.GradientTape() as tape:
            # Train the model on the states and updated Q-values
            q_values = model(state_sample)

            # Apply the masks to the Q-values to get the Q-value for action taken
            q_action = tf.reduce_sum(tf.multiply(q_values, masks), axis=1)
            # Calculate loss between new Q-value and old Q-value
            loss = loss_function(updated_q_values, q_action)

        # Backpropagation
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        # update the the target network with new weights
        model_target.set_weights(model.get_weights())
        # Log details
        template = "running reward: {:.2f} at episode {}, frame count {}"
        print(template.format(running_reward, episode_count, frame_count))

        # Limit the state and reward history
        if len(rewards_history) > max_memory_length:
            del rewards_history[:1]
            del state_history[:1]
            del state_next_history[:1]
            del action_history[:1]
            del done_history[:1]

        # Update running reward to check condition for solving
        episode_reward_history.append(episode_reward)
        if len(episode_reward_history) > 100:
            del episode_reward_history[:1]

        running_reward = np.mean(episode_reward_history)

        episode_count += 1

        if running_reward > 40:  # Condition to consider the task solved
            print("Solved at episode {}!".format(episode_count))

        valid_cards = self._rule.get_valid_cards_from_obs(obs)
        card = np.random.choice(np.flatnonzero(valid_cards))
        return card


## The Jass loop

In [41]:
def local_arena():
    arena = Arena(nr_games_to_play=1000)
    arena.set_players(AgentReinforcement(), AgentReinforcement(), AgentReinforcement(), AgentReinforcement())
    arena.play_all_games()

    count = 0
    for i in range(arena.nr_games_played):
        if arena.points_team_0[i] > arena.points_team_1[i]:
            count = count + 1

    print("Team 0 : " + str(arena.points_team_0.sum()) + ": Games won : " + str(count))
    print("Team 1 : " + str(arena.points_team_1.sum()) + ": Games won : " + str(arena.nr_games_played - count))

local_arena()

TypeError: cannot unpack non-iterable NoneType object